In [8]:
! pip install gradio_client
! pip install groq
! pip install transformers

In [5]:
import itertools
import re
import numpy as np
import pandas as pd
# import openai
import pandas as pd
import itertools
# import streamlit as st
import pickle
# Anna's imports
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gradio_client import Client

from groq import Groq


In [6]:
df = pd.read_parquet('Halved-DF.parquet.gzip')

# combinations_of_two -> working correctly

##### input: 
user input = single string with ingredients separated by commas and a space

##### output: 
ingredients_combinations = all possible combinations of 2 as a list of tuples (so if it's 4 ingredients, a list of 6 tuples and nothing more)

In [114]:
# Updated 5/21/2024 by TJ

def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

    '''
    The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
    NOTE FOR FRONT-END: The output of this function is the input for data_query()
    
    Input: 
    ingredients_input (from user input) =  single string with ingredients separated by commas and a space
    
    Output: 
    ingredients_combinations (to data_query) = a list of tuples, containing all possible combinations of 2 (so if it's 4 ingredients, a list of 6 tuples and nothing more)

    '''

    # split into list of individual ingredients
    ingredients_list = []
    ingredients = re.split('\s|,', ingredients_input) 
    [ingredients_list.append(ingredient) for ingredient in ingredients if ingredient != '']
    
    # create list of all ingredient combinations
    ingredients_combinations = []
    combinations = itertools.combinations(ingredients_list, 2)
    ingredients_combinations = list(combinations)

    return ingredients_combinations

## Testing combinations_of_two

In [111]:
# 4 ingredients -> 1 list of 6 unique tuples
ingredients_input = 'chicken, garlic, salt, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
ingredients_combinations

[('chicken', 'garlic'),
 ('chicken', 'salt'),
 ('chicken', 'tomato'),
 ('garlic', 'salt'),
 ('garlic', 'tomato'),
 ('salt', 'tomato')]

In [112]:
# 6 ingredients -> 1 list of 15 unique tuples
ingredients_input = 'tomato, beef, onion, salt, garlic, pepper'

ingredients_combinations = combinations_of_two(ingredients_input)
ingredients_combinations

[('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'salt'),
 ('tomato', 'garlic'),
 ('tomato', 'pepper'),
 ('beef', 'onion'),
 ('beef', 'salt'),
 ('beef', 'garlic'),
 ('beef', 'pepper'),
 ('onion', 'salt'),
 ('onion', 'garlic'),
 ('onion', 'pepper'),
 ('salt', 'garlic'),
 ('salt', 'pepper'),
 ('garlic', 'pepper')]

In [115]:
# missing spaces and commas
ingredients_input = 'tomato,beef,  onion chicken'

ingredients_combinations = combinations_of_two(ingredients_input)
ingredients_combinations

[('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'chicken'),
 ('beef', 'onion'),
 ('beef', 'chicken'),
 ('onion', 'chicken')]

# data_query
Issues:
- output is not including all the combinations

In [ ]:
def data_query(ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    '''
    Input: 
    ingredients_combinations (from combinations_of_two) = a list of tuples, containing all possible combinations of 2 as  
    
    Output: 
    df_comb (to muse_comb) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    '''
    
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})
    df_comb = pd.DataFrame(data)
    return df_comb

In [65]:
# # trying to understand / re-create
def data_query(ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    '''
    Input: 
    ingredients_combinations (from combinations_of_two) = a list of tuples, containing all possible combinations of 2 as  
    
    Output: 
    df_comb (to muse_comb) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    '''
    
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            print(score)
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})
    print(data)

    df_comb = pd.DataFrame(data)
    return df_comb

## Testing data_query

In [66]:
# 6 tuples

ingredients_combinations = [('chicken', 'garlic'),
 ('chicken', 'salt'),
 ('chicken', 'tomato'),
 ('garlic', 'salt'),
 ('garlic', 'tomato'),
 ('salt', 'tomato')]

df_comb = data_query(ingredients_combinations)
df_comb

[1.18008113]
[1.43539727]
[]
[1.55331592]
[]
[]
[{'Combination': ('chicken', 'garlic'), 'Score': array([1.18008113])}, {'Combination': ('chicken', 'salt'), 'Score': array([1.43539727])}, {'Combination': ('garlic', 'salt'), 'Score': array([1.55331592])}]


,Combination,Score
0,"(chicken, garlic)",[1.1800811339314616]
1,"(chicken, salt)",[1.4353972677154387]
2,"(garlic, salt)",[1.5533159218950603]


In [68]:
# 15 tuples

ingredients_combinations = [('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'salt'),
 ('tomato', 'garlic'),
 ('tomato', 'pepper'),
 ('beef', 'onion'),
 ('beef', 'salt'),
 ('beef', 'garlic'),
 ('beef', 'pepper'),
 ('onion', 'salt'),
 ('onion', 'garlic'),
 ('onion', 'pepper'),
 ('salt', 'garlic'),
 ('salt', 'pepper'),
 ('garlic', 'pepper')]

df_comb = data_query(ingredients_combinations)
df_comb

[]
[]
[]
[]
[]
[1.34074608]
[1.36673683]
[1.13859574]
[1.18191367]
[2.06707352]
[1.42354091]
[1.49490488]
[1.55331592]
[3.35645049]
[1.19027345]
[{'Combination': ('beef', 'onion'), 'Score': array([1.34074608])}, {'Combination': ('beef', 'salt'), 'Score': array([1.36673683])}, {'Combination': ('beef', 'garlic'), 'Score': array([1.13859574])}, {'Combination': ('beef', 'pepper'), 'Score': array([1.18191367])}, {'Combination': ('onion', 'salt'), 'Score': array([2.06707352])}, {'Combination': ('onion', 'garlic'), 'Score': array([1.42354091])}, {'Combination': ('onion', 'pepper'), 'Score': array([1.49490488])}, {'Combination': ('salt', 'garlic'), 'Score': array([1.55331592])}, {'Combination': ('salt', 'pepper'), 'Score': array([3.35645049])}, {'Combination': ('garlic', 'pepper'), 'Score': array([1.19027345])}]


,Combination,Score
0,"(beef, onion)",[1.3407460751179292]
1,"(beef, salt)",[1.3667368322711522]
2,"(beef, garlic)",[1.1385957417039734]
3,"(beef, pepper)",[1.1819136702926785]
4,"(onion, salt)",[2.0670735182650013]
5,"(onion, garlic)",[1.4235409132992736]
6,"(onion, pepper)",[1.4949048817870405]
7,"(salt, garlic)",[1.5533159218950603]
8,"(salt, pepper)",[3.3564504877818235]
9,"(garlic, pepper)",[1.1902734549399014]


# muse_comb

In [61]:
def muse_comb(df_comb): ###If this takes too long, consider taking the nested calculate_sum(array) outside of the function
    '''
     the function calculates the sum of the "Score" values and returns the three combinations with the largest sums
     OUTPUT: [['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'flour', 'sugar']]

     NOTE FOR FRONT-END: The return is a list of lists so access the values by indexing e.g. output[0]

                         The output of this function is the input for the recipe generator

                         We might need a function to convert each lists into strings if
                         the recipe generator doesn't do this automatically.
                         
    Input: 
    df_comb (from data_query) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    Output:
    ingredients_list (to recipe_generator) = a list of 3 lists, containing the 3 ingredients combinations with highest scores
    
    '''

    def calculate_sum(array):
        return sum(array)

    def ingredients_to_lists(lists):
        ingredients_list = []
        for i in range(3):
            tmp_list = []
            for x in lists[i]:
                tmp_list.append(x[0])
                tmp_list.append(x[1])
            ingredients_list.append(list(set(tmp_list)))

        return ingredients_list

    for i in range(len(df_comb)):
        df_comb["Sum"] = df_comb["Score"].apply(calculate_sum)

    max_values = df_comb.nlargest(3, "Sum")

    max_values = max_values["Combination"].reset_index(drop=True)

    ingredients_lists = ingredients_to_lists(max_values)

    return ingredients_lists

## Testing muse_comb

In [62]:
# 6 tuples

ingredients_combinations = [('chicken', 'garlic'),
 ('chicken', 'salt'),
 ('chicken', 'tomato'),
 ('garlic', 'salt'),
 ('garlic', 'tomato'),
 ('salt', 'tomato')]

df_comb = data_query(ingredients_combinations)

ingredients_lists = muse_comb(df_comb)
ingredients_lists

[]
[{'Combination': ('chicken', 'garlic'), 'Score': array([1.18008113])}, {'Combination': ('chicken', 'salt'), 'Score': array([1.43539727])}, {'Combination': ('garlic', 'salt'), 'Score': array([1.55331592])}]


[['s', 'g', 'a'], ['h', 's', 'a', 'c'], ['a', 'h', 'g', 'c']]

In [67]:
# 15 tuples

ingredients_combinations = [('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'salt'),
 ('tomato', 'garlic'),
 ('tomato', 'pepper'),
 ('beef', 'onion'),
 ('beef', 'salt'),
 ('beef', 'garlic'),
 ('beef', 'pepper'),
 ('onion', 'salt'),
 ('onion', 'garlic'),
 ('onion', 'pepper'),
 ('salt', 'garlic'),
 ('salt', 'pepper'),
 ('garlic', 'pepper')]

df_comb = data_query(ingredients_combinations)

ingredients_lists = muse_comb(df_comb)
ingredients_lists

[]
[]
[]
[]
[]
[1.34074608]
[1.36673683]
[1.13859574]
[1.18191367]
[2.06707352]
[1.42354091]
[1.49490488]
[1.55331592]
[3.35645049]
[1.19027345]
[{'Combination': ('beef', 'onion'), 'Score': array([1.34074608])}, {'Combination': ('beef', 'salt'), 'Score': array([1.36673683])}, {'Combination': ('beef', 'garlic'), 'Score': array([1.13859574])}, {'Combination': ('beef', 'pepper'), 'Score': array([1.18191367])}, {'Combination': ('onion', 'salt'), 'Score': array([2.06707352])}, {'Combination': ('onion', 'garlic'), 'Score': array([1.42354091])}, {'Combination': ('onion', 'pepper'), 'Score': array([1.49490488])}, {'Combination': ('salt', 'garlic'), 'Score': array([1.55331592])}, {'Combination': ('salt', 'pepper'), 'Score': array([3.35645049])}, {'Combination': ('garlic', 'pepper'), 'Score': array([1.19027345])}]


[['p', 's', 'e', 'a'], ['o', 'n', 's', 'a'], ['s', 'g', 'a']]

# recipe_generator

In [69]:
def recipe_generator(lists):
    
    '''
    Input: 
    ingredients_list (from muse_comb) = a list of 3 lists, containing the 3 ingredients combinations with highest score
    
    Output: recipe_list (to final_recipe) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
    
    '''
    api_key = "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"
    client = Groq(
    api_key=api_key
    )
    recipe_list = []

    if len(lists) == 1:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {lists} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    else:
      for i in range(len(lists)):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {lists[i]} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    return recipe_list

In [70]:
ingredients_lists = [['tomato', 'chicken', 'garlic'], ['chicken', 'onion', 'salt'], ['onion', 'salt', 'garlic']]

recipe_list = recipe_generator(ingredients_lists)
recipe_list

APIConnectionError: Connection error.

# final_recipes

In [137]:
def final_recipes(recipe_list, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't
    """
    This evaluates whether the score of a recipe passes or fails the threshold.
    If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
    INPUT: Output of the recipe generator function
    NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
                        old version for this function to still work.
                        optimized_gptrecipe() and scoring_model() must be replaced with the actual functions
    
    Input: 
    recipe_list (from recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
    scores (from code in app.py) = list of 3 integers, containing scores for each recipe
    model (from get_model in app.py) = object 
    
    Output: final_recipes = 3 final recipes as a list of 3 dictionaries with 'title', 'ingredients', 'directions' as keys
    
    """
    final_recipes = {"Title": [], "Ingredients": [], "Directions": []}
    threshold = 0.5

    for i in range(len(recipe_dict)):
        if scores[i] >= threshold:
            final_recipes["Title"].append(recipe_dict[i]['title'])
            final_recipes["Ingredients"].append(recipe_dict[i]['ingredients'])
            final_recipes["Directions"].append(recipe_dict[i]['directions'])
        else:
            n = 0
            tmp_recipe = {
                "Title":recipe_dict[i]['title'],
                "Ingredients":recipe_dict[i]['ingredients'],
                "Directions":recipe_dict[i]['directions']
                         }
            last_recipe = {"title":[],
                           "ingredients":[],
                           "directions":[]
                         }
            while n < 3:
                new_recipe = recipe_generator([tmp_recipe["Ingredients"]]) ###<=== insert actual recipe generator
                new_score = model.predict_proba([new_recipe[0]['directions']]) ###<=== insert the actual scoring model function here
                if new_score[0][1] >= threshold:
                    final_recipes["Title"].append(new_recipe[0]["title"])
                    final_recipes["Ingredients"].append(new_recipe[0]["ingredients"])
                    final_recipes["Directions"].append(new_recipe[0]["directions"])
                    break  # Exit loop if the new recipe passes the threshold
                else:
                    last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
                    n += 1
            else: # Add the last generated recipe if the loop completes without finding a passing recipe
                final_recipes["Title"].append(last_recipe["title"][0][0])
                final_recipes["Ingredients"].append(last_recipe["ingredients"][0][0])
                final_recipes["Directions"].append(last_recipe["directions"][0][0])
                break  # Exit the outer loop to prevent an unending loop

    return final_recipes

In [15]:
def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model

model = get_model()

In [139]:
combinations = combinations_of_two('chicken, mustard, coconut, orange, gravy')
combinations

[('chicken', 'mustard'),
 ('chicken', 'coconut'),
 ('chicken', 'orange'),
 ('chicken', 'gravy'),
 ('mustard', 'coconut'),
 ('mustard', 'orange'),
 ('mustard', 'gravy'),
 ('coconut', 'orange'),
 ('coconut', 'gravy'),
 ('orange', 'gravy')]

In [151]:
df_comb = data_query(combinations)
df_comb

,Combination,Score
0,"(chicken, mustard)",[1.0066298927435484]
1,"(chicken, coconut)",[1.0035599592711963]
2,"(chicken, orange)",[1.0035599592711963]
3,"(chicken, gravy)",[1.0004900257988445]
4,"(mustard, coconut)",[1.001709310046983]
5,"(mustard, orange)",[1.0036608667953153]
6,"(mustard, gravy)",[1.0192917213569856]
7,"(coconut, orange)",[1.0069021295275749]


In [150]:
ingredients_lists = muse_comb(df_comb)
ingredients_lists

[['g', 'r', 'u', 'm'], ['r', 'c', 'o'], ['u', 'm', 'c', 'h']]

In [148]:
ingredients_lists = 

recipe_dicts = recipe_generator(ingredients_lists)
recipe_dicts

[{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprin

In [33]:
def get_scores(recipe_dicts):

    scores = []
    recipe_direction = []

    for recipe in recipe_dicts:
        if 'directions' in recipe:
            recipe_direction.append(recipe['directions'])
        else:
            recipe_direction.append("")

    for direction in recipe_direction:
        scores.append(model.predict_proba([direction])[0][1])

    return scores

In [17]:

recipe_dicts = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

get_scores([{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}])

[0.5040546079674728, 0.5040292302849937, 0.5123256028930298]

In [34]:
main_dict = final_recipes(recipe_dicts, scores, model)
print(main_dict)

NameError: name 'final_recipes' is not defined

In [ ]:
fake_scores = [0.5241604487770766, 0.3045873107190447, 0.1205374202247903]

In [ ]:
test_dict = final_recipes(recipe_dicts, fake_scores, model)
print(test_dict)